## self-implemented segment tree after watching YouTube video

### trees are implemented in array

In [1]:
from typing import List

In [67]:
class SegmentTree:
    
    def __init__(self, array: List[int]) -> None:
        """
        for range minimum query,
        low and high is for the real array,
        pos is for segment tree, 
        trick is that 2 * pos + 1 is the left child and 2 * pos + 2 is the right child
        the leaf node condition is low == high
        """
        self.array = [array[i] if i < len(array) else float("inf") \
                      for i in range(self._get_next_power_of_two(len(array)))]
        self.segTree = [float("inf") for i in range(2*len(self.array)-1)]
        self._construct_tree(0, len(self.array)-1, 0)
        print("*** successfully construct segment tree ***")
        print("*** array ***")
        print(self.array)
        print("*** segment tree ***")
        print(self.segTree)
    
    
    def min_query(self, qlow: int, qhigh: int) -> int:
        return self._range_min_query(qlow, qhigh, 0, len(self.array)-1, 0)
        
    
    def _get_next_power_of_two(self, n: int) -> int:
        # this for power of 2
        if n and not(n & (n - 1)):
            return n
        count = 0
        while n != 0:
            n >>= 1
            count += 1
        return 1 << count

    
    def _construct_tree(self, low: int, high: int, pos: int) -> None:
#         print("low, high, pos", low, high, pos)
        # leaf node condition
        if low == high:
            self.segTree[pos] = self.array[low]
            return
        mid = (low + high) // 2
        self._construct_tree(low, mid, 2*pos+1)
        self._construct_tree(mid+1, high, 2*pos+2)
        self.segTree[pos] = min(self.segTree[2*pos+1], self.segTree[2*pos+2])
    
    
    def _range_min_query(self, qlow: int, qhigh: int, low: int, high: int, pos: int) -> int:
#         print("qlow, qhigh, low, high, pos", qlow, qhigh, low, high, pos)
        # total overlap
        if qlow <= low and qhigh >= high:
            return self.segTree[pos]
        # no overlap
        if qlow > high or qhigh < low:
            return float("inf")
        # partial overlap
        mid = (low + high) // 2
        return min(self._range_min_query(qlow, qhigh, low, mid, 2*pos+1),\
                   self._range_min_query(qlow, qhigh, mid+1, high, 2*pos+2))

In [69]:
segTree = SegmentTree([-1, 2, 4, 0])
segTree.min_query(0, 3), segTree.min_query(1, 2), segTree.min_query(2, 3)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 0]
*** segment tree ***
[-1, -1, 0, -1, 2, 4, 0]


(-1, 2, 0)

In [70]:
segTree = SegmentTree([-1, 2, 4, 0, -2, -3])
segTree.min_query(0, 4), segTree.min_query(2, 4), segTree.min_query(3, 4), segTree.min_query(3, 5)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 0, -2, -3, inf, inf]
*** segment tree ***
[-3, -1, -3, -1, 0, -3, inf, -1, 2, 4, 0, -2, -3, inf, inf]


(-2, -2, -2, -3)

In [71]:
from operator import add, mul

In [72]:
class SegmentTreeGeneral:
    
    def __init__(self, array: List[int], operation: str) -> None:
        """
        to generalize the implementation above
        sum: 0
        mul: 1
        min: float("inf")
        max: -float("inf")
        """
        self.support = {"sum": (0, add), "mul": (1, mul), \
                        "min": (float("inf"), min), \
                        "max": (-float("inf"), max)}
        assert operation in self.support
        self.defaultVal = self.support[operation][0]
        self.func = self.support[operation][1]
        self.array = [array[i] if i < len(array) else self.defaultVal \
                      for i in range(self._get_next_power_of_two(len(array)))]
        self.segTree = [self.defaultVal for i in range(2*len(self.array)-1)]
        self._construct_tree(0, len(self.array)-1, 0)
        print("*** successfully construct segment tree ***")
        print("*** array ***")
        print(self.array)
        print("*** segment tree ***")
        print(self.segTree)
    
    
    def query(self, qlow: int, qhigh: int) -> int:
        return self._range_query(qlow, qhigh, 0, len(self.array)-1, 0)
        
    
    def _get_next_power_of_two(self, n: int) -> int:
        # this for power of 2
        if n and not(n & (n - 1)):
            return n
        count = 0
        while n != 0:
            n >>= 1
            count += 1
        return 1 << count

    
    def _construct_tree(self, low: int, high: int, pos: int) -> None:
        # leaf node condition
        if low == high:
            self.segTree[pos] = self.array[low]
            return
        mid = (low + high) // 2
        self._construct_tree(low, mid, 2*pos+1)
        self._construct_tree(mid + 1, high, 2*pos+2)
        self.segTree[pos] = self.func(self.segTree[2*pos+1], self.segTree[2*pos+2])
    
    
    def _range_query(self, qlow: int, qhigh: int, low: int, high: int, pos: int) -> int:
        # total overlap
        if qlow <= low and qhigh >= high:
            return self.segTree[pos]
        # no overlap
        if qlow > high or qhigh < low:
            return self.defaultVal
        # partial overlap
        mid = (low + high) // 2
        return self.func(self._range_query(qlow, qhigh, low, mid, 2*pos+ 1),\
                         self._range_query(qlow, qhigh, mid+1, high, 2*pos+2))

In [73]:
segTree = SegmentTreeGeneral([-1, 2, 4, 0], "sum")
segTree.query(0, 3), segTree.query(1, 2), segTree.query(2, 3)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 0]
*** segment tree ***
[5, 1, 4, -1, 2, 4, 0]


(5, 6, 4)

In [74]:
segTree = SegmentTreeGeneral([-1, 2, 4, 0, -2, -3], "mul")
segTree.query(0, 4), segTree.query(2, 4), segTree.query(3, 4), segTree.query(3, 5)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 0, -2, -3, 1, 1]
*** segment tree ***
[0, 0, 6, -2, 0, 6, 1, -1, 2, 4, 0, -2, -3, 1, 1]


(0, 0, 0, 0)

In [75]:
segTree = SegmentTreeGeneral([-1, 2, 4, 5, -2, -3], "mul")
segTree.query(0, 4), segTree.query(2, 4), segTree.query(3, 4), segTree.query(3, 5)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 5, -2, -3, 1, 1]
*** segment tree ***
[-240, -40, 6, -2, 20, 6, 1, -1, 2, 4, 5, -2, -3, 1, 1]


(80, -40, -10, 30)

### lazy propagation for range update

In [78]:
class SegmentTree:
    
    def __init__(self, array: List[int]) -> None:
        """
        implement lazy propagation to incrementally update segment tree for minimum range query
        the update is incremental (+) only, because max(elements + delta) = max(elements) + delta
        that is the condition the update needs to meet to do lazy propagation
        another case might be sum(elements * delta) = sum(elements) * delta
        update function needs to be implemented from scratch and query function needs to be updated
        """
        self.array = [array[i] if i < len(array) else float("inf") \
                      for i in range(self._get_next_power_of_two(len(array)))]
        self.segTree = [float("inf") for i in range(2*len(self.array)-1)]
        self.lazyTree = [0 for i in range(len(self.segTree))]
        self._construct_tree(0, len(self.array)-1, 0)
        print("*** successfully construct segment tree ***")
        print("*** array ***")
        print(self.array)
        print("*** segment tree ***")
        print(self.segTree)
    
    
    def min_query(self, qlow: int, qhigh: int) -> int:
        return self._range_min_query_lazy(qlow, qhigh, 0, len(self.array)-1, 0)
    
    
    def update(self, uplow: int, uphigh: int, delta: int) -> None:
        self._range_update_lazy(uplow, uphigh, delta, 0, len(self.array)-1, 0)
        
    
    def _get_next_power_of_two(self, n: int) -> int:
        # this for power of 2
        if n and not(n & (n - 1)):
            return n
        count = 0
        while n != 0:
            n >>= 1
            count += 1
        return 1 << count

    
    def _construct_tree(self, low: int, high: int, pos: int) -> None:
        # leaf node condition
        if low == high:
            self.segTree[pos] = self.array[low]
            return
        mid = (low + high) // 2
        self._construct_tree(low, mid, 2*pos+1)
        self._construct_tree(mid+1, high, 2*pos+2)
        self.segTree[pos] = min(self.segTree[2*pos+1], self.segTree[2*pos+2])
    
    
    def _range_min_query_lazy(self, qlow: int, qhigh: int, low: int, high: int, pos: int) -> int:
        # make sure the node is up-to-date and propagate lazily if needed
        if self.lazyTree[pos] != 0:
            self.segTree[pos] += self.lazyTree[pos]
            # leaf node check
            if low != high:
                self.lazyTree[2*pos+1] += self.lazyTree[pos]
                self.lazyTree[2*pos+2] += self.lazyTree[pos]
            self.lazyTree[pos] = 0       
        # total overlap
        if qlow <= low and qhigh >= high:
            return self.segTree[pos]
        # no overlap
        if qlow > high or qhigh < low:
            return float("inf")
        # partial overlap
        mid = (low + high) // 2
        return min(self._range_min_query_lazy(qlow, qhigh, low, mid, 2*pos+1),\
                   self._range_min_query_lazy(qlow, qhigh, mid+1, high, 2*pos+2))
    
    def _range_update_lazy(self, uplow: int, uphigh: int, delta: int, low: int, high: int, pos: int) -> None:
        # make sure all propagation is done at current node and propagate to children if needed
        if self.lazyTree[pos] != 0:
            self.segTree[pos] += self.lazyTree[pos]
            # leaf node check
            if low != high:
                self.lazyTree[2*pos+1] += self.lazyTree[pos]
                self.lazyTree[2*pos+2] += self.lazyTree[pos]
            self.lazyTree[pos] = 0
        # no overlap
        if uplow > high or uphigh < low:
            return
        # total overlap, update current node and propagate lazily to children
        if uplow <= low and uphigh >= high:
            self.segTree[pos] += delta
            if low != high:
                self.lazyTree[2*pos+1] += delta
                self.lazyTree[2*pos+2] += delta
            return
        # partial overlap
        mid = (low + high) // 2
        self._range_update_lazy(uplow, uphigh, delta, low, mid, 2*pos+1)
        self._range_update_lazy(uplow, uphigh, delta, mid+1, high, 2*pos+2)
        # after the update function is executed, the node at pos is up-to-date, this is important
        self.segTree[pos] = min(self.segTree[2*pos+1], self.segTree[2*pos+2])

In [79]:
segTree = SegmentTree([-1, 2, 4, 1, 7, 1, 3, 2])
segTree.update(0, 3, 3)
segTree.update(0, 3, 1)
segTree.update(0, 0, 2)
segTree.min_query(3, 5)

*** successfully construct segment tree ***
*** array ***
[-1, 2, 4, 1, 7, 1, 3, 2]
*** segment tree ***
[-1, -1, 1, -1, 1, 1, 2, -1, 2, 4, 1, 7, 1, 3, 2]


1

In [80]:
segTree = SegmentTree([2, 3, -1, 4])
segTree.update(0, 3, 2)
segTree.update(2, 2, 4)
segTree.min_query(1, 2)

*** successfully construct segment tree ***
*** array ***
[2, 3, -1, 4]
*** segment tree ***
[-1, 2, -1, 2, 3, -1, 4]


5

## 307. Range Sum Query - Mutable

### self-written solution, passed all tests, but a bit slow

In [89]:
class SegmentTreeSum:
    
    def __init__(self, array: List[int]) -> None:
        """
        no lazy propagation update
        """
        self.array = [array[i] if i < len(array) else 0 \
                      for i in range(self._get_next_power_of_two(len(array)))]
        self.segTree = [0 for i in range(2*len(self.array)-1)]
        self._construct_tree(0, len(self.array)-1, 0)
    
    
    def sum_query(self, qlow: int, qhigh: int) -> int:
        return self._range_sum_query(qlow, qhigh, 0, len(self.array)-1, 0)
    
    def update(self, index: int, val: int) -> int:
        self._update_single(index, val-self.array[index], 0, len(self.array)-1, 0)
        self.array[index] = val
        
    
    def _get_next_power_of_two(self, n: int) -> int:
        # this for power of 2
        if n and not(n & (n - 1)):
            return n
        count = 0
        while n != 0:
            n >>= 1
            count += 1
        return 1 << count

    
    def _construct_tree(self, low: int, high: int, pos: int) -> None:
        # leaf node condition
        if low == high:
            self.segTree[pos] = self.array[low]
            return
        mid = (low + high) // 2
        self._construct_tree(low, mid, 2*pos+1)
        self._construct_tree(mid+1, high, 2*pos+2)
        self.segTree[pos] = self.segTree[2*pos+1] + self.segTree[2*pos+2]
    
    
    def _range_sum_query(self, qlow: int, qhigh: int, low: int, high: int, pos: int) -> int:
        # total overlap
        if qlow <= low and qhigh >= high:
            return self.segTree[pos]
        # no overlap
        if qlow > high or qhigh < low:
            return 0
        # partial overlap
        mid = (low + high) // 2
        return self._range_sum_query(qlow, qhigh, low, mid, 2*pos+1) + \
               self._range_sum_query(qlow, qhigh, mid+1, high, 2*pos+2)
    
    
    def _update_single(self, index: int, delta: int, low: int, high: int, pos: int) -> None:
        self.segTree[pos] += delta
        # leaf node check
        if low != high:
            mid = (low + high) // 2
            if low <= index <= mid:
                self._update_single(index, delta, low, mid, 2*pos+1)
            else:
                self._update_single(index, delta, mid+1, high, 2*pos+2)

In [90]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.segTree = SegmentTreeSum(nums)
        

    def update(self, index: int, val: int) -> None:
        self.segTree.update(index, val)
        

    def sumRange(self, left: int, right: int) -> int:
        return self.segTree.sum_query(left, right)

In [91]:
numArray = NumArray([1, 3, 5])
print(numArray.sumRange(0, 2)) # return 1 + 3 + 5 = 9
numArray.update(1, 2) # nums = [1, 2, 5]
print(numArray.sumRange(0, 2)) # return 1 + 2 + 5 = 8

9
8


### self-written bottom-up iterative approach for range query and single update after checking solution

In [94]:
class NumArray:

    def __init__(self, nums: List[int]):
        """
        the trick is leave the first element in segment tree empty
        then the left and right child would be 2*pos and 2*pos+1
        parent would be pos // 2
        """
        self.length = n = len(nums)
        self.segTree = [0] * (2*n)
        # fill in leaf nodes first
        for i in range(n, 2*n):
            self.segTree[i] = nums[i-n]
        # fill in parent nodes bottom-up
        for j in range(n-1, 0, -1):
            self.segTree[j] = self.segTree[2*j] + self.segTree[2*j+1]

        
    # two approach here
    def _update_1(self, index: int, val: int) -> None:
        # this approach tells us how to identify right/left child and is more general
        pos = index + self.length
        self.segTree[pos] = val
        while pos > 1:
            if pos % 2 == 0:
                left, right = pos, pos+1
            else:
                left, right = pos-1, pos
            # update parent 
            self.segTree[pos//2] = self.segTree[left] + self.segTree[right]
            pos //= 2  
            
    
    def _update_2(self, index: int, val: int) -> None:
        # special approach for this question
        pos = index + self.length
        delta = val - self.segTree[pos]
        while pos > 0:
            # update current
            self.segTree[pos] += delta
            pos //= 2    
        
    
    def update(self, index: int, val: int) -> None: 
        return self._update_2(index, val)
    
    # this is really genius!!! Can't fully understand
    def sumRange(self, left: int, right: int) -> int:
        """
        the idea here is to shrink [left, right] iteratively to fully cover the range
        the details can refer to http://codeforces.com/blog/entry/18051
        """
        left += self.length
        right += self.length
        ret = 0
        while left <= right:
            if left % 2 == 1:
                ret += self.segTree[left]
                left += 1
            if right % 2 == 0:
                ret += self.segTree[right]
                right -= 1
            left //= 2
            right //= 2
        return ret

In [95]:
numArray = NumArray([1, 3, 5])
print(numArray.sumRange(0, 2)) # return 1 + 3 + 5 = 9
numArray.update(1, 2) # nums = [1, 2, 5]
print(numArray.sumRange(0, 2)) # return 1 + 2 + 5 = 8

9
8


## 303. Range Sum Query - Immutable

### self-written solution

In [99]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.cumsum = [0] * (len(nums)+1)
        for i in range(len(nums)):
            self.cumsum[i+1] = self.cumsum[i] + nums[i]

    def sumRange(self, left: int, right: int) -> int:
        return self.cumsum[right+1] - self.cumsum[left]

In [101]:
numArray = NumArray([-2, 0, 3, -5, 2, -1])
print(numArray.sumRange(0, 2)) # return (-2) + 0 + 3 = 1
print(numArray.sumRange(2, 5)) # return 3 + (-5) + 2 + (-1) = -1
print(numArray.sumRange(0, 5)) # return (-2) + 0 + 3 + (-5) + 2 + (-1) = -3

1
-1
-3


## 218. The Skyline Problem

### self-written priority queue solution after watching YouTube video and checking solution in discussion area

In [103]:
import heapq
from collections import defaultdict

In [107]:
class Solution218:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        """
        ideally the process runs as following:
        for each point:
            add new buildings starting at point
            remove old buildings ending at point
            check the currently highest building
        however priority queue doesn't support removing operation
        the workaround is to remove the high buildings ending at the point that will affect the skyline,
        and leave those lower buildings (which doesn't affect the skyline yet) in the priority queue
        """
        # points of interests
        points = sorted(set([b[0] for b in buildings] + [b[1] for b in buildings]))
        point2building = defaultdict(list)
        for b in buildings:
            point2building[b[0]].append(b)
        skyline_tracker = []
        last_highest = 0
        res = []
        for point in points:
            # add buildings, negate cause heapq is a minimum queue
            for b in point2building[point]:
                _, end, height = b
                heapq.heappush(skyline_tracker, (-height, end)) # end is just for records
            # pop tall buildings, <= is because of the workaround (otherwise == is enough)
            while skyline_tracker and skyline_tracker[0][1] <= point:
                heapq.heappop(skyline_tracker)
            # important: heap root is at [0]
            curr_highest = -skyline_tracker[0][0] if skyline_tracker else 0
            if last_highest != curr_highest:
                res.append([point, curr_highest])
                last_highest = curr_highest
        return res

In [108]:
solver_218 = Solution218()
solver_218.getSkyline([[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]])

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [109]:
solver_218 = Solution218()
solver_218.getSkyline([[0,2,3],[2,5,3]])

[[0, 3], [5, 0]]

### self-translated iterative approach for range update without lazy propagation and single query

#### many bitwise tricks learned from the post

In [137]:
class SegmentTreeMax:
    
    def __init__(self, length: int) -> None:
        """
        without initializing
        support range update and single query
        [left, right) implementation
        refer to http://codeforces.com/blog/entry/18051
        """
        self.segTree = [0 for _ in range(2*length)]
        self.length = length
    
    def range_update(self, left, right, value):
        left += self.length
        right += self.length
        while left < right:
            if left & 1:
                self.segTree[left] = max(self.segTree[left], value)
                left += 1
            if right & 1:
                right -= 1
                self.segTree[right] = max(self.segTree[right], value)
            left >>= 1
            right >>= 1
    
    def query_single(self, index):
        res = 0
        index += self.length
        while index > 0:
            res = max(res, self.segTree[index])
            index >>= 1
        return res
    
    def push(self):
        for i in range(1, self.length):
            self.segTree[i<<1] = max(self.segTree[i<<1], self.segTree[i])
            self.segTree[i<<1|1] = max(self.segTree[i<<1|1], self.segTree[i])
            self.segTree[i] = 0

### self-written segment tree solution

In [140]:
class Solution218:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        # points of interests
        points = sorted(set([b[0] for b in buildings] + [b[1] for b in buildings]))
        point2idx = {point: idx for idx, point in enumerate(points)}
        segment_tree = SegmentTreeMax(len(points))
        for start, end, height in buildings:
            # important that we should not take end into account of the skyline (take one building as an example)
            # the update here is open interval
            segment_tree.range_update(point2idx[start], point2idx[end], height)
#         # the update is not pushed to leaf layer yet
#         print(segment_tree.segTree[-len(points):])
#         segment_tree.push()
#         print(segment_tree.segTree[-len(points):])
        res = []
        last_highest = 0
        for idx, point in enumerate(points):
            curr_highest = segment_tree.query_single(idx)
            if last_highest != curr_highest:
                res.append([point, curr_highest])
                last_highest = curr_highest
        return res

In [141]:
solver_218 = Solution218()
solver_218.getSkyline([[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]])

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [142]:
solver_218 = Solution218()
solver_218.getSkyline([[0,2,3],[2,5,3]])

[[0, 3], [5, 0]]

### self-written recursive segment tree solution with lazy propagation, slow but passed all tests

In [174]:
class SegmentTreeMax:
    
    def __init__(self, length: int) -> None:
        self.length = length
        self.segTree = [0 for i in range(2*self._get_next_power_of_two(length)-1)]
        self.lazyTree = [0 for i in range(len(self.segTree))]
    
    
    def query_single(self, index: int) -> int:
        return self._single_max_query_lazy(index, 0, self.length-1, 0)
    
    
    def range_update(self, uplow: int, uphigh: int, val: int) -> None:
        self._range_update_lazy(uplow, uphigh, val, 0, self.length-1, 0)
        
    
    def _get_next_power_of_two(self, n: int) -> int:
        # this for power of 2
        if n and not(n & (n - 1)):
            return n
        count = 0
        while n != 0:
            n >>= 1
            count += 1
        return 1 << count
    
    
    def _single_max_query_lazy(self, index: int, low: int, high: int, pos: int) -> int:
        # make sure the node is up-to-date and propagate lazily if needed
        if self.lazyTree[pos] != 0:
            self.segTree[pos] = max(self.segTree[pos], self.lazyTree[pos])
            # leaf node check
            if low != high:
                self.lazyTree[2*pos+1] = max(self.lazyTree[2*pos+1], self.lazyTree[pos])
                self.lazyTree[2*pos+2] = max(self.lazyTree[2*pos+2], self.lazyTree[pos])
            self.lazyTree[pos] = 0
        # reach the leaf
        if low == high:
            return self.segTree[pos]
        # partial overlap
        mid = (low + high) // 2
        if index <= mid:
            return self._single_max_query_lazy(index, low, mid, 2*pos+1)
        return self._single_max_query_lazy(index, mid+1, high, 2*pos+2)
    
    def _range_update_lazy(self, uplow: int, uphigh: int, val: int, low: int, high: int, pos: int) -> None:
        # make sure all propagation is done at current node and propagate to children if needed
        if self.lazyTree[pos] != 0:
            self.segTree[pos] = max(self.segTree[pos], self.lazyTree[pos])
            # leaf node check
            if low != high:
                self.lazyTree[2*pos+1] = max(self.lazyTree[2*pos+1], self.lazyTree[pos])
                self.lazyTree[2*pos+2] = max(self.lazyTree[2*pos+2], self.lazyTree[pos])
            self.lazyTree[pos] = 0
        # no overlap
        if uplow > high or uphigh < low:
            return
        # total overlap, update current node and propagate lazily to children
        if uplow <= low and uphigh >= high:
            self.segTree[pos] = max(self.segTree[pos], val)
            if low != high:
                self.lazyTree[2*pos+1] = max(self.lazyTree[2*pos+1], val)
                self.lazyTree[2*pos+2] = max(self.lazyTree[2*pos+2], val)
            return
        # partial overlap
        mid = (low + high) // 2
        self._range_update_lazy(uplow, uphigh, val, low, mid, 2*pos+1)
        self._range_update_lazy(uplow, uphigh, val, mid+1, high, 2*pos+2)
        # after the update function is executed, the node at pos is up-to-date, this is important
        self.segTree[pos] = max(self.segTree[2*pos+1], self.segTree[2*pos+2])

In [175]:
class Solution218:
    def getSkyline(self, buildings: List[List[int]]) -> List[List[int]]:
        # points of interests
        points = sorted(set([b[0] for b in buildings] + [b[1] for b in buildings]))
        point2idx = {point: idx for idx, point in enumerate(points)}
        segment_tree = SegmentTreeMax(len(points))
        for start, end, height in buildings:
            # the update here is closed interval
            segment_tree.range_update(point2idx[start], point2idx[end]-1, height)
        res = []
        last_highest = 0
        for idx, point in enumerate(points):
            curr_highest = segment_tree.query_single(idx)
            if last_highest != curr_highest:
                res.append([point, curr_highest])
                last_highest = curr_highest
        return res

In [176]:
solver_218 = Solution218()
solver_218.getSkyline([[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]])

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [177]:
solver_218 = Solution218()
solver_218.getSkyline([[0,2,3],[2,5,3]])

[[0, 3], [5, 0]]

## 850. Rectangle Area II

### self-writtenn line sweep solution after checking answers

In [ ]:
class Solution850:
    def rectangleArea(self, rectangles: List[List[int]]) -> int:
        """
        vertical line sweep horizontally
        """
        START, END = 0, 1
        intervals = [(x1, START, y1, y2) for x1, y1, x2, y2 in rectangles]
        intervals.extend([(x2, END, y1, y2) for x1, y1, x2, y2 in rectangles])
        intervals.sort()
        active_intervals, area, prev_x = [], 0, inntervals[0][0]
        for x, flag, y1, y2 in intervals:
            if x != prev_x:
            if flag == START:
                intervals.append((y1, y2))